# Scraper Monkey

* This project performs web-scraping on Amazon.com, Bestbuy.com using Python to find best price for Samsung TVs. 

<img src="Slide1.jpeg">

<img src="Slide2.jpeg">

In [222]:
#import all the necessary libraries:
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np
import requests
import csv
from splinter import Browser
from sqlalchemy import create_engine

In [223]:
#To configure webdriver to use Chrome browser, we have to set the path to chromedriver

##Windows Machine
# executable_path = {"executable_path": "chromedriver.exe"}
# driver = Browser("chrome", **executable_path, headless=False)
#driver= webdriver.Chrome(executable_path)

## Mac Machine
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
driver = Browser('chrome', **executable_path, headless=False)

In [224]:
product_price_df = pd.DataFrame()
product_price_df["sold_by"] = np.nan
product_price_df["seller_name"] = np.nan
product_price_df["product_Brand"] = np.nan
product_price_df["product_name"] = np.nan
product_price_df["product_model"] = np.nan
product_price_df["product_sku"]=np.nan
product_price_df["product_price"] = np.nan
product_price_df["product_category"]= np.nan
product_price_df["product_image"]= np.nan
product_price_df["product_url"]= np.nan

keyword = "Samsung TV"

query_URL_BestBuy_keyword= "https://www.bestbuy.com/site/searchpage.jsp?_dyncharset=UTF-8&id=pcat17071&iht=y&keys=keys&ks=960&list=n&sc=Global&st="+keyword+"&type=page&usc=All%20Categories"

#query_URL_FryElec_keyword= "https://www.frys.com/search?query_search=&cat=-68520&nearbyStoreName=false&pType=pDisplay&fq=a%20Regular%20Items-m10099998%20Samsung&start=0&cat=-68520&from=0&to=99&isKeyword=true"


## Extract

In [225]:
def extract_products_from_amazon(product_price_df, keyword):
    #Url of the page to be scraped
    request_url = f"https://www.amazon.com/s?k={keyword}"
#   print(request_url)
    driver.visit(request_url)

#     for x in range(1, 4):
    response = driver.html
    soup = bs(response)
    for product_doc in soup.findAll('div',attrs={'class':'s-include-content-margin s-border-bottom'}):
        try:
            name=product_doc.find('span', attrs={'class':'a-size-medium a-color-base a-text-normal'})
            product_name = name.text
            if("Samsung" in product_name ):
                price=product_doc.find('span', attrs={'class':'a-offscreen'})
                product_price = price.text
                product_price = float(product_price.split("$")[1].replace(",",""))
               
                image= product_doc.find('img', attrs={'class':'s-image'})
                image_url = image['src']
                
                product_page_link= product_doc.find('a', attrs={'class':'a-link-normal'})
#                 print(product_link)
                product_page_url = product_page_link['href']
                
                product_page_url = 'https://www.amazon.com'+product_page_url
#                 print(product_page_url)
                data = {
                    'product_name': product_name,
                    'product_price': product_price,
#                   'product_model': product_model,
                    'sold_by': 'Amazon',
                    'seller_name': 'Amazon',
                    'product_Brand': 'Samsung TV',
                    'product_category': 'Electronics',
                    'product_image': image_url,
                    'product_sku':"not_available",
                    'Model_SKU Name':'not_available',
                    'product_url': product_page_url 
                }
                product_price_df = product_price_df.append(data, ignore_index=True)
        except Exception as e:
            print(f"Oops!  exception occured - continue")
#             browser.click_link_by_partial_text('Next')
    print(product_price_df.count())
#     product_price_df.head()
    return product_price_df

def transformProductPriceAmazon(product_price_df):
# product_price_df

    for row in product_price_df.itertuples():
        try:
            name = product_price_df['product_name'][row.Index]
#             print(name)

            pattern = re.compile(r'([UQ]N[0-9a-zA-Z]{5,12})', re.IGNORECASE)
            result = pattern.findall(name)
            for model in result:
    #                 product_model = model
                product_price_df['product_model'][row.Index] =  model

        except IndexError:
            print(f"Oops!  That was no valid number !!")

    return product_price_df 

In [226]:
def scrapProductPriceBestBuy(query_URL_BestBuy_keyword):
    driver.visit(query_URL_BestBuy_keyword)
    content = driver.html
    soup = bs(content)
    row_index=1
    for bestBuyHTML in soup.findAll('li', attrs={'class':'sku-item'}):
        name=bestBuyHTML.find('div', attrs={'class':'sku-title'})
        model = bestBuyHTML.find('div', attrs={'class':'sku-model'})   
        price= bestBuyHTML.find('div', attrs={'class':'priceView-customer-price'})
        image= bestBuyHTML.find('img', attrs={'class':'product-image'})
        image_url = image['src']
        
        product_page_link= bestBuyHTML.find('a', attrs={'class':'image-link'})
        product_page_url = product_page_link['href']
        
        product_page_url = 'https://www.bestbuy.com'+product_page_url
        
        model = model.text
        TV_model_name = name.text
        TV_Model = model.split(" ")[0].split(":")[1]
        TV_SKU = model.split(" ")[1].split(":")[1]
        TV_price = price.span.text
        #TV_price = TV_price.split("Your")[0]
        product_price_df.loc[row_index,"sold_by"] = "Best Buy"
        product_price_df.loc[row_index,"seller_name"] = "Best Buy"
        product_price_df.loc[row_index,"product_Brand"] = "Samsung TV"
        product_price_df.loc[row_index,"product_category"] = "Electronics"
        product_price_df.loc[row_index,"product_name"] = TV_model_name.strip()
        product_price_df.loc[row_index,"product_price"] = TV_price
        product_price_df.loc[row_index,"Model_SKU Name"] = model.strip()
        product_price_df.loc[row_index,"product_image"] = image_url.strip()
        product_price_df.loc[row_index,"product_url"] = product_page_url.strip()
        row_index = row_index + 1 
    return product_price_df

def transformProductPriceBestBuy(product_price_df):
    for row in product_price_df.itertuples():
        TV_model_name = product_price_df["Model_SKU Name"][row.Index]
        TV_Model = TV_model_name.split(" ")[0].split(":")[1]
        TV_SKU = TV_model_name.split(" ")[1].split(":")[1]
        product_price_df["product_model"][row.Index] = TV_Model
        product_price_df["product_sku"][row.Index] = TV_SKU
        product_price_df["product_price"][row.Index] = float(product_price_df["product_price"][row.Index].split("$")[1].replace(",",""))
    return product_price_df

In [227]:
# Extract Best Buy product price data
product_price_df= scrapProductPriceBestBuy(query_URL_BestBuy_keyword)
# transform Best Buy product model name to model and SKU
product_price_df = transformProductPriceBestBuy(product_price_df)
# product_price_df.head()

/Users/m631230/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/m631230/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/m631230/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [229]:
# Extract Amazon product price data

product_price_df = extract_products_from_amazon(product_price_df, keyword)
product_price_df = transformProductPriceAmazon(product_price_df.copy())
# product_price_df.head()

Oops!  exception occured - continue
Oops!  exception occured - continue
Oops!  exception occured - continue
Oops!  exception occured - continue
sold_by             36
seller_name         36
product_Brand       36
product_name        36
product_model       24
product_sku         36
product_price       36
product_category    36
product_image       36
product_url         36
Model_SKU Name      36
dtype: int64


In [230]:
product_price_df.count()

sold_by             36
seller_name         36
product_Brand       36
product_name        36
product_model       36
product_sku         36
product_price       36
product_category    36
product_image       36
product_url         36
Model_SKU Name      36
dtype: int64

In [231]:
product_price_summary_df = product_price_df[["product_name","product_model","seller_name","product_price"]]

product_price_summary_pivot_df = pd.pivot_table(product_price_summary_df,values="product_price",index=["product_model"],columns=["seller_name"],aggfunc=np.min)
product_price_summary_pivot_df= product_price_summary_pivot_df.dropna()
product_price_summary_pivot_df.head()

product_price_df= product_price_df[["product_model","product_name","product_price", "product_image","product_url"]]
product_price_summary = product_price_summary_pivot_df.merge(product_price_df,on="product_model",how="left")
# product_price_summary.rename(columns={
#     'Amazon':"amazon_price",
#     'Best Buy':"best_buy_price"
# })
product_price_summary.head(20)

,product_model,Amazon,Best Buy,product_name,product_price,product_image,product_url
0,QN65Q60RAFXZA,997.99,999.99,"Samsung - 65"" Class - LED - Q60 Series - 2160p...",999.99,https://pisces.bbystatic.com/image2/BestBuy_US...,https://www.bestbuy.com/site/samsung-65-class-...
1,QN65Q60RAFXZA,997.99,999.99,Samsung QN65Q60RAFXZA Flat 65-Inch QLED 4K Q60...,997.99,https://m.media-amazon.com/images/I/716SrHQrbj...,https://www.amazon.com/Samsung-QN65Q60RAFXZA-F...
2,QN65Q70RAFXZA,1197.99,1199.99,"Samsung - 65"" Class - LED - Q70 Series - 2160p...",1199.99,https://pisces.bbystatic.com/image2/BestBuy_US...,https://www.bestbuy.com/site/samsung-65-class-...
3,QN65Q70RAFXZA,1197.99,1199.99,Samsung QN65Q70RAFXZA Flat 65-Inch QLED 4K Q70...,1197.99,https://m.media-amazon.com/images/I/91eWjdLgiJ...,https://www.amazon.com/Samsung-QN65Q70RAFXZA-6...
4,UN50NU6900FXZA,339.99,329.99,"Samsung - 50"" Class - LED - NU6900 Series - 21...",329.99,https://pisces.bbystatic.com/image2/BestBuy_US...,https://www.bestbuy.com/site/samsung-50-class-...
5,UN50NU6900FXZA,339.99,329.99,"Samsung Electronics 4K Smart LED TV (2018), 50...",339.99,https://m.media-amazon.com/images/I/A1ZEwxqfJV...,https://www.amazon.com/Samsung-Electronics-Sma...
6,UN55RU7100FXZA,447.99,449.99,"Samsung - 55"" Class - LED - 7 Series - 2160p -...",449.99,https://pisces.bbystatic.com/image2/BestBuy_US...,https://www.bestbuy.com/site/samsung-55-class-...
7,UN55RU7100FXZA,447.99,449.99,Samsung UN55RU7100FXZA Flat 55-Inch 4K UHD 7 S...,447.99,https://m.media-amazon.com/images/I/91GMSrYPaH...,https://www.amazon.com/Samsung-UN55RU7100FXZA-...
8,UN55RU8000FXZA,547.99,549.99,"Samsung - 55"" Class - LED - 8 Series - 2160p -...",549.99,https://pisces.bbystatic.com/image2/BestBuy_US...,https://www.bestbuy.com/site/samsung-55-class-...
9,UN55RU8000FXZA,547.99,549.99,Samsung UN55RU8000FXZA Flat 55-Inch 4K 8 Serie...,547.99,https://m.media-amazon.com/images/I/918xc0BpEp...,https://www.amazon.com/Samsung-UN55RU8000FXZA-...


In [201]:
rds_connection_string = "postgres:{password}@localhost:5432/Product_PriceDB"
# rds_connection_string = "postgres:Ucb@2019@10.105.216.109:5432/Product_PriceDB"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [202]:
engine.table_names()

['product_price']

In [203]:
product_price_summary.to_sql(name='product_price', con=engine, if_exists='replace', index=False)

In [204]:
pd.read_sql_query('select * from product_price', con=engine).head()

,product_model,Amazon,Best Buy,product_name,product_price,product_image,product_url
0,QN65Q60RAFXZA,997.99,999.99,"Samsung - 65"" Class - LED - Q60 Series - 2160p...",999.99,https://pisces.bbystatic.com/image2/BestBuy_US...,https://www.bestbuy.com/site/samsung-65-class-...
1,QN65Q60RAFXZA,997.99,999.99,Samsung QN65Q60RAFXZA Flat 65-Inch QLED 4K Q60...,997.99,https://m.media-amazon.com/images/I/716SrHQrbj...,https://www.amazon.com/Samsung-QN65Q60RAFXZA-F...
2,QN75Q70RAFXZA,1997.99,1999.99,"Samsung - 75"" Class - LED - Q70 Series - 2160p...",1999.99,https://pisces.bbystatic.com/image2/BestBuy_US...,https://www.bestbuy.com/site/samsung-75-class-...
3,QN75Q70RAFXZA,1997.99,1999.99,Samsung QN75Q70RAFXZA Flat 75-Inch QLED 4K Q70...,1997.99,https://m.media-amazon.com/images/I/91eWjdLgiJ...,https://www.amazon.com/Samsung-QN75Q70RAFXZA-F...
4,UN55RU7100FXZA,447.99,449.99,"Samsung - 55"" Class - LED - 7 Series - 2160p -...",449.99,https://pisces.bbystatic.com/image2/BestBuy_US...,https://www.bestbuy.com/site/samsung-55-class-...
